In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

import sys
sys.path.append("../")
from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

In [2]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jonas-sievers (ipe). Use `wandb login --relogin` to force relogin


True

In [3]:
#Get data 
cwd = os.path.normpath(os.path.dirname(os.path.dirname(os.getcwd())))
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

#Select only 3 User for testing
df_user10 = df[['User10', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', 'User10_lag_24hrs']]
df_user11 = df[['User11', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', 'User11_lag_24hrs']]
df_user12 = df[['User12', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', 'User12_lag_24hrs']]
df_array = [df_user10, df_user11, df_user12]
df_array[1].head(3)

,User11,temp,rhum,wspd,PC1,hour sin,hour cos,User11_lag_24hrs
Date,,,,,,,,
2012-07-08 00:00:00,0.312,9.8,85.0,0.0,-2.453691,0.000000,1.000000,0.667
2012-07-08 01:00:00,0.263,9.8,85.0,0.0,-2.453691,0.258819,0.965926,0.316
2012-07-08 02:00:00,0.257,9.8,85.0,0.0,-2.453691,0.500000,0.866025,0.356


In [4]:
#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user1{idx}'], y_train[f'user1{idx}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user1{idx}'], y_val[f'user1{idx}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user1{idx}'], y_test[f'user1{idx}'] = dh.prepare_data(test_sequences, batch_size)

In [13]:
#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

# Initialize wandb
run = wandb.init(project='keras-integration',
                 config={
                     "learning_rate": 0.005,
                     "epochs": max_epochs,
                     "batch_size": batch_size,
                     "loss_function": "tf.keras.losses.MeanSquaredError()",
                     "architecture": "Dense",
                     "dataset": "Load"
                 })
config = wandb.config  # We'll use this to configure our model

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
callbacks=[early_stopping, timing_callback, custom_callback, WandbCallback()]

In [14]:
tf.keras.backend.clear_session()
dense_results = pd.DataFrame(columns=['architecture', 'Loss@User10','std@User10', 'Loss@User11','std@User11', 'Loss@User12','std@User12'])

In [15]:
#dense_architectures to test: 

#dense_architectures - tested: L1_U4, L2_U4, L3_U4, L4_U4, L5_U4, L1_U8, L2_U8, L3_U8, L4_U8, L5_U8, L1_U16, L2_U16, L3_U16, L4_U16, L5_U16, L1_U32
#L2_U32, L3_U32, L1_U64, L2_U64, L1_U128, L2_U128, L1_U256, L2_U256, L1_U512, L2_U512

#Dense Hyperparameter
dense_architecture = "L2_U512"
dense_layers = 2
dense_units = 32

#dense_results = pd.DataFrame(columns=['architecture', 'Loss@User10', 'Loss@User11', 'Loss@User12'])

dense_all_results = pd.DataFrame(columns=["user", "architecture", "mse"])
#For each of the 3 user
for idx in range(3):
    print("User: ", idx)
    for round in range(3):
        #print("Round: ", round)
        dense_model = m1.build_dense_model(X_train[f'user1{idx}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=config.batch_size)
        wandb.watch(dense_model)
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user1{idx}'],
            y_train = y_train[f'user1{idx}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user1{idx}'], 
            y_val=y_val[f'user1{idx}'], 
            X_test=X_test[f'user1{idx}'], 
            y_test=y_test[f'user1{idx}'], 
            callbacks=callbacks, 
            user=f'user1{idx}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

new_row = {
    'architecture': dense_all_results["architecture"][0],
    'Loss@User10': dense_all_results[dense_all_results["user"]=="user10"]["mse"].mean(),
    'std@User10' : dense_all_results[dense_all_results["user"]=="user10"]["mse"].std(),
    'Loss@User11': dense_all_results[dense_all_results["user"]=="user11"]["mse"].mean(),
    'std@User11' : dense_all_results[dense_all_results["user"]=="user11"]["mse"].std(),
    'Loss@User12': dense_all_results[dense_all_results["user"]=="user12"]["mse"].mean(),
    'std@User12' : dense_all_results[dense_all_results["user"]=="user12"]["mse"].std(),
}
dense_results.loc[len(dense_results)] = new_row
print(dense_results)
run.finish()

User:  0


Error: wandb.watch only works with pytorch, couldn't import torch.

In [34]:
lstm_results = pd.DataFrame(columns=['architecture', 'Loss@User10','std@User10', 'Loss@User11','std@User11', 'Loss@User12','std@User12'])

In [36]:
#LSTM Hyperparameter
lstm_architecture = "Bi_L2_U20"
lstm_layers = 2
lstm_units = 8

#lstm_results = pd.DataFrame(columns=['architecture', 'Loss@User10', 'Loss@User11', 'Loss@User12'])

lstm_all_results = pd.DataFrame(columns=["user", "architecture", "mse"])
#For each of the 3 user
for idx in range(3):
    print("User: ", idx)
    for round in range(3):
        print("Round: ", round)
        #lstm_model = m1.build_lstm_model(X_train[f'user1{idx}'], horizon, lstm_layers, lstm_units, batch_size)
        lstm_model = m1.build_bilstm_model(X_train[f'user1{idx}'], horizon, lstm_layers, lstm_units, batch_size)
        lstm_histroy, lstm_user_results = mh.compile_fit_evaluate_model(
            model=lstm_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user1{idx}'],
            y_train = y_train[f'user1{idx}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user1{idx}'], 
            y_val=y_val[f'user1{idx}'], 
            X_test=X_test[f'user1{idx}'], 
            y_test=y_test[f'user1{idx}'], 
            callbacks=callbacks, 
            user=f'user1{idx}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from lstm_user_results to lstm_results
        lstm_all_results = pd.merge(lstm_all_results, lstm_user_results, how='outer')   

new_row = {
    'architecture': lstm_all_results["architecture"][0],
    'Loss@User10': lstm_all_results[lstm_all_results["user"]=="user10"]["mse"].mean(),
    'std@User10' : lstm_all_results[lstm_all_results["user"]=="user10"]["mse"].std(),
    'Loss@User11': lstm_all_results[lstm_all_results["user"]=="user11"]["mse"].mean(),
    'std@User11' : lstm_all_results[lstm_all_results["user"]=="user11"]["mse"].std(),
    'Loss@User12': lstm_all_results[lstm_all_results["user"]=="user12"]["mse"].mean(),
    'std@User12' : lstm_all_results[lstm_all_results["user"]=="user12"]["mse"].std(),
}
lstm_results.loc[len(lstm_results)] = new_row
print(lstm_results)

User:  0
Round:  0
Round:  1
Round:  2
User:  1
Round:  0
Round:  1
Round:  2
User:  2
Round:  0
Round:  1
Round:  2
  architecture  Loss@User10  std@User10  Loss@User11  std@User11  Loss@User12  \
0    Bi_L2_U20     0.019668    0.003406     0.017234    0.001019      0.01073   

   std@User12  
0    0.001039  
